# 1 Apple store application rating prediction


In [341]:
# Some intro text? 

All needed imports for the whole project can be found below

In [342]:
# place imports here
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam, SGD
from keras.utils.np_utils import to_categorical

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

First we load in our dataset and look if the data is loaded correctly.

In [343]:
dataset = pd.read_csv('../dataset/AppleStore.csv')
dataset.head()

,Unnamed: 0,id,track_name,size_bytes,currency,price,rating_count_tot,rating_count_ver,user_rating,user_rating_ver,ver,cont_rating,prime_genre,sup_devices.num,ipadSc_urls.num,lang.num,vpp_lic
0,1,281656475,PAC-MAN Premium,100788224,USD,3.99,21292,26,4.0,4.5,6.3.5,4+,Games,38,5,10,1
1,2,281796108,Evernote - stay organized,158578688,USD,0.00,161065,26,4.0,3.5,8.2.2,4+,Productivity,37,5,23,1
2,3,281940292,"WeatherBug - Local Weather, Radar, Maps, Alerts",100524032,USD,0.00,188583,2822,3.5,4.5,5.0.0,4+,Weather,37,5,3,1
3,4,282614216,"eBay: Best App to Buy, Sell, Save! Online Shop...",128512000,USD,0.00,262241,649,4.0,4.5,5.10.0,12+,Shopping,37,5,9,1
4,5,282935706,Bible,92774400,USD,0.00,985920,5320,4.5,5.0,7.5.1,4+,Reference,37,5,45,1


In [344]:
# also look at the different types of the data
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7197 entries, 0 to 7196
Data columns (total 17 columns):
Unnamed: 0          7197 non-null int64
id                  7197 non-null int64
track_name          7197 non-null object
size_bytes          7197 non-null int64
currency            7197 non-null object
price               7197 non-null float64
rating_count_tot    7197 non-null int64
rating_count_ver    7197 non-null int64
user_rating         7197 non-null float64
user_rating_ver     7197 non-null float64
ver                 7197 non-null object
cont_rating         7197 non-null object
prime_genre         7197 non-null object
sup_devices.num     7197 non-null int64
ipadSc_urls.num     7197 non-null int64
lang.num            7197 non-null int64
vpp_lic             7197 non-null int64
dtypes: float64(3), int64(9), object(5)
memory usage: 955.9+ KB


Then we take a look if there are null values in our dataset and if this might have any consequences.

In [345]:
dataset.isnull().sum()

Unnamed: 0          0
id                  0
track_name          0
size_bytes          0
currency            0
price               0
rating_count_tot    0
rating_count_ver    0
user_rating         0
user_rating_ver     0
ver                 0
cont_rating         0
prime_genre         0
sup_devices.num     0
ipadSc_urls.num     0
lang.num            0
vpp_lic             0
dtype: int64

Since there are no null values, this shouldn't have any consequences for the rest of this project.

# 2 Data exploration

Now take a deeper look at the data

In [346]:
dataset.describe()

,Unnamed: 0,id,size_bytes,price,rating_count_tot,rating_count_ver,user_rating,user_rating_ver,sup_devices.num,ipadSc_urls.num,lang.num,vpp_lic
count,7197.000000,7.197000e+03,7.197000e+03,7197.000000,7.197000e+03,7197.000000,7197.000000,7197.000000,7197.000000,7197.000000,7197.000000,7197.000000
mean,4759.069612,8.631310e+08,1.991345e+08,1.726218,1.289291e+04,460.373906,3.526956,3.253578,37.361817,3.707100,5.434903,0.993053
std,3093.625213,2.712368e+08,3.592069e+08,5.833006,7.573941e+04,3920.455183,1.517948,1.809363,3.737715,1.986005,7.919593,0.083066
min,1.000000,2.816565e+08,5.898240e+05,0.000000,0.000000e+00,0.000000,0.000000,0.000000,9.000000,0.000000,0.000000,0.000000
25%,2090.000000,6.000937e+08,4.692275e+07,0.000000,2.800000e+01,1.000000,3.500000,2.500000,37.000000,3.000000,1.000000,1.000000
50%,4380.000000,9.781482e+08,9.715302e+07,0.000000,3.000000e+02,23.000000,4.000000,4.000000,37.000000,5.000000,1.000000,1.000000
75%,7223.000000,1.082310e+09,1.819249e+08,1.990000,2.793000e+03,140.000000,4.500000,4.500000,38.000000,5.000000,8.000000,1.000000
max,11097.000000,1.188376e+09,4.025970e+09,299.990000,2.974676e+06,177050.000000,5.000000,5.000000,47.000000,5.000000,75.000000,1.000000


Some things we notice here are:
1. More than 50% of apps are free and most apps are low prices even tough there are some high priced apps.
2. There will be both perfect rated apps and non rated apps in our model.
3. There are some columns that will 100% be not usefull during this project or are not clear what they mean (also after reading the documentation of the dataset. These Columns are the following: sup_devices.num, ipadSc_urls.num, lang.num, vpp_lic, track_name and Unnamed: 0. These colums will be removed at part 3 "Removeing noise data"

Now lets look if there are colums with some unique values like user rating etc.
This will be done using the pandas .unique() function

In [347]:
dataset.currency.unique()

array(['USD'], dtype=object)

Only one currency makes it so that we dont have to convert the prices to one and the same currency

In [348]:
dataset.user_rating.unique()

array([ 4. ,  3.5,  4.5,  5. ,  3. ,  2. ,  2.5,  0. ,  1.5,  1. ])

The rating of an app is between 0 and 5 with increments of 0.5 so there are no ratings like 3.66

In [349]:
dataset.prime_genre.unique()

array(['Games', 'Productivity', 'Weather', 'Shopping', 'Reference',
       'Finance', 'Music', 'Utilities', 'Travel', 'Social Networking',
       'Sports', 'Business', 'Health & Fitness', 'Entertainment',
       'Photo & Video', 'Navigation', 'Education', 'Lifestyle',
       'Food & Drink', 'News', 'Book', 'Medical', 'Catalogs'], dtype=object)

In [350]:
dataset.cont_rating.unique()

array(['4+', '12+', '17+', '9+'], dtype=object)

In [351]:
dataset.price.unique()

array([   3.99,    0.  ,    0.99,    9.99,    4.99,    7.99,    2.99,
          1.99,    5.99,   12.99,   21.99,  249.99,    6.99,   74.99,
         19.99,    8.99,   24.99,   13.99,   14.99,   16.99,   47.99,
         11.99,   59.99,   15.99,   27.99,   17.99,  299.99,   49.99,
         23.99,   20.99,   39.99,   99.99,   29.99,   34.99,   18.99,
         22.99])

In [352]:
# 3 Removing noise data renaming and normalizing

In part 2 we found colums that will be dropped from the dataframe. These columns are: sup_devices.num, ipadSc_urls.num, lang.num, vpp_lic.
Also because there is 1 currency we don't need the currency column because the price column is enough.
After dropping we check if the actually are dropped by using the .head() function to look at the dataset

In [353]:
dataset = dataset.drop(["Unnamed: 0","track_name", "currency","sup_devices.num","ipadSc_urls.num","lang.num","vpp_lic","id"], axis=1)
dataset.head()

,size_bytes,price,rating_count_tot,rating_count_ver,user_rating,user_rating_ver,ver,cont_rating,prime_genre
0,100788224,3.99,21292,26,4.0,4.5,6.3.5,4+,Games
1,158578688,0.00,161065,26,4.0,3.5,8.2.2,4+,Productivity
2,100524032,0.00,188583,2822,3.5,4.5,5.0.0,4+,Weather
3,128512000,0.00,262241,649,4.0,4.5,5.10.0,12+,Shopping
4,92774400,0.00,985920,5320,4.5,5.0,7.5.1,4+,Reference


In [354]:
# saving this dataset to our dataset folder so we can use this in the future.
dataset.to_csv("../dataset/AppleStoreWithoutNoise.csv", index=False, encoding='utf8')

In [355]:

copy = dataset

copy = copy.drop(copy[copy.user_rating < 1].index)
dataset = copy

Next thing we have to do is normalize some values and create catergories for non numeric values. 

As of now there are a lot of different app sizes which have a large spread. (obviously because its given in bytes) there are two options to normalize these. We can either: 
1. Put the different sizes into categories like : < 1mb or > 100mb etc.
2. Normalize the bytes to kb or even mb

Next there is the price, earlier we saw the prices range from 0 - 299.99. We have two options here:
1. Normalize by dividing by a own determined number.
2. use MinMax normalization which normalizes all numbers to be between 0 and 1
Option 2 should be the most optimal here for our model.

rating_count_tot has two options to normalize
1. Normalize by dividing by a own determined number.
2. use MinMax normalization which normalizes all numbers to be between 0 and 1
The best option here : TBA

rating_count_ver is not certain if this column can be used for our model so we will not normalize for now, but leave it in just in case.

user_rating is an easy scale that can be normalized using the MinMax normalization which is also the most optimal option for our model.

user_rating_ver is not certain if this column can be used for our model so we will not normalize for now, but leave it in just in case.

version is not certain if this column can be used for our model so we will not normalize for now, but leave it in just in case.

cont_rating and prime_genre both have multiple different possible values which are all text. To use these field we need to convert these someway to numbers. To do this we can us the function "get_dummies()" from pandas utils. This function will add the amount of unique values in columns to the dataset and mark the right feature value with a 1 and the rest with a 0.

In [356]:
# size_bytes normalization
# after looking at the data we can see the range is between 576kb and 3.72529Gb. 
# So the first thing we can do is convert the size_bytes to size_100kilo_bytes by dividing 1024 (b -> kb) and 100 (kb ->100kb)

dataset['size_100kilo_bytes'] = dataset['size_bytes']/102400

In [357]:
# price normalization

mms = MinMaxScaler()
dataset['price_mms'] = mms.fit_transform(dataset[['price']])

In [358]:
# rating_count_tot normalization
# TODO

In [359]:
# user_rating normalization

dataset['user_rating_mms'] = mms.fit_transform(dataset[['user_rating']])

In [360]:
#user_rating count 

dataset['rating_count_mms'] = mms.fit_transform(dataset[['rating_count_tot']])

In [361]:
# cont_rating and prime_genre dummies

dataset_dummies = pd.get_dummies(dataset[['cont_rating','prime_genre']])
dataset_dummies.head()

,cont_rating_12+,cont_rating_17+,cont_rating_4+,cont_rating_9+,prime_genre_Book,prime_genre_Business,prime_genre_Catalogs,prime_genre_Education,prime_genre_Entertainment,prime_genre_Finance,...,prime_genre_News,prime_genre_Photo & Video,prime_genre_Productivity,prime_genre_Reference,prime_genre_Shopping,prime_genre_Social Networking,prime_genre_Sports,prime_genre_Travel,prime_genre_Utilities,prime_genre_Weather
0,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,1,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
2,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
4,0,0,1,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0


In [362]:
# Check if all values are now normalized correct
dataset.describe()

,size_bytes,price,rating_count_tot,rating_count_ver,user_rating,user_rating_ver,size_100kilo_bytes,price_mms,user_rating_mms,rating_count_mms
count,6.268000e+03,6268.000000,6.268000e+03,6268.000000,6268.000000,6268.000000,6268.000000,6268.000000,6268.000000,6268.000000
mean,2.057430e+08,1.821977,1.480381e+04,528.607371,4.049697,3.735801,2009.209099,0.006073,0.762424,0.004976
std,3.526341e+08,6.128238,8.098468e+04,4196.699812,0.726943,1.399040,3443.692860,0.020428,0.181736,0.027225
min,5.898240e+05,0.000000,1.000000e+00,0.000000,1.000000,0.000000,5.760000,0.000000,0.000000,0.000000
25%,5.171942e+07,0.000000,7.800000e+01,6.000000,4.000000,3.500000,505.072500,0.000000,0.750000,0.000026
50%,1.021292e+08,0.000000,5.125000e+02,37.000000,4.500000,4.500000,997.355000,0.000000,0.875000,0.000172
75%,1.884774e+08,2.990000,3.963500e+03,176.000000,4.500000,4.500000,1840.600000,0.009967,0.875000,0.001332
max,4.025970e+09,299.990000,2.974676e+06,177050.000000,5.000000,5.000000,39316.110000,1.000000,1.000000,1.000000


# Setting up and training a model

In [389]:
# TODO

from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.pipeline import Pipeline
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from keras.wrappers.scikit_learn import KerasClassifier


In [390]:
dataset['is_not_free'] = dataset['price'].apply(lambda x: 1 if x > 0 else 0)
dataset.head()

,size_bytes,price,rating_count_tot,rating_count_ver,user_rating,user_rating_ver,ver,cont_rating,prime_genre,size_100kilo_bytes,price_mms,user_rating_mms,rating_count_mms,is_not_free
0,100788224,3.99,21292,26,4.0,4.5,6.3.5,4+,Games,984.26,0.0133,0.750,0.007157,1
1,158578688,0.00,161065,26,4.0,3.5,8.2.2,4+,Productivity,1548.62,0.0000,0.750,0.054145,0
2,100524032,0.00,188583,2822,3.5,4.5,5.0.0,4+,Weather,981.68,0.0000,0.625,0.063396,0
3,128512000,0.00,262241,649,4.0,4.5,5.10.0,12+,Shopping,1255.00,0.0000,0.750,0.088158,0
4,92774400,0.00,985920,5320,4.5,5.0,7.5.1,4+,Reference,906.00,0.0000,0.875,0.331438,0


In [391]:
def categorize_rating(x):
    if x <= 4:
        return 0
    else:
        return 1

In [392]:
X = pd.concat([copy[['is_not_free', 'rating_count_mms']], dataset_dummies], axis=1).values
Y = dataset['user_rating']
Y = Y.apply(categorize_rating).values

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)
len(X_train)
X.shape

(6268, 29)

In [393]:
def model_builder_sigmoid():
    model = Sequential()
    model.add(Dense(1, input_shape=(29,), activation='sigmoid'))
    model.compile(SGD(lr=0.5), 
                  loss='binary_crossentropy', 
                  metrics=['accuracy'])
    #model.fit(X_train, Y_train, epochs=25)
    return model

In [394]:
model = KerasClassifier(build_fn=model_builder_sigmoid,
                        epochs=25,
                        verbose=0)
cv = KFold(3, shuffle=True)
scores = cross_val_score(model, X, Y, cv=cv)
print("The cross validation accuracy is {:0.4f} ± {:0.4f}".format(scores.mean(), scores.std()))

The cross validation accuracy is 0.5900 ± 0.0129


In [395]:
from sklearn.metrics import accuracy_score, confusion_matrix


model = model_builder_sigmoid()
model.fit(X_train, Y_train, epochs=25)


y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)

print("The Accuracy score on the Train set is:\t{:0.3f}".format(accuracy_score(Y_train, y_train_pred.round())))
print("The Accuracy score on the Test set is:\t{:0.3f}".format(accuracy_score(Y_test, y_test_pred.round())))

Epoch 1/25
5014/5014 [==============================] - 1s - loss: 0.6815 - acc: 0.5716     
Epoch 2/25
5014/5014 [==============================] - 0s - loss: 0.6752 - acc: 0.5822     
Epoch 3/25
5014/5014 [==============================] - 0s - loss: 0.6717 - acc: 0.5955     
Epoch 4/25
5014/5014 [==============================] - 0s - loss: 0.6706 - acc: 0.5985     
Epoch 5/25
5014/5014 [==============================] - 0s - loss: 0.6705 - acc: 0.5991     
Epoch 6/25
5014/5014 [==============================] - 0s - loss: 0.6687 - acc: 0.5961     
Epoch 7/25
5014/5014 [==============================] - 0s - loss: 0.6693 - acc: 0.5911     
Epoch 8/25
5014/5014 [==============================] - 0s - loss: 0.6697 - acc: 0.5963     
Epoch 9/25
5014/5014 [==============================] - 0s - loss: 0.6682 - acc: 0.5971     
Epoch 10/25
5014/5014 [==============================] - 0s - loss: 0.6685 - acc: 0.5947     
Epoch 11/25
5014/5014 [==============================] - 0s - loss: 0

In [ ]:
copy.describe()

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.pipeline import Pipeline
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from keras.wrappers.scikit_learn import KerasClassifier

In [ ]:
#model = Sequential()
#model.add(Dense(29, input_dim=29, kernel_initializer='normal', activation='relu'))
#model.add(Dense(14, kernel_initializer='normal', activation='relu'))
#model.add(Dense(1, kernel_initializer='normal'))
#model.compile(Adam(lr=0.05), 'mean_squared_error')

# define the model
def model_builder():
# create model
	model = Sequential()
	model.add(Dense(1, input_dim=29, kernel_initializer='normal', activation='relu'))
#model.add(Dense(6, kernel_initializer='normal', activation='relu'))
#model.add(Dense(1, kernel_initializer='normal'))
# Compile model
	model.compile(Adam(lr=0.01), loss='mean_squared_error', metrics=['accuracy'])
	return model


In [ ]:
X = pd.concat([copy[['price_mms', 'rating_count_mms']], dataset_dummies], axis=1).values
X.shape

In [ ]:
Y = dataset['user_rating_mms'].values
Y.shape
Y

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
# split the data into train and test with a 20% test size


In [ ]:
seed =7
np.random.seed(seed)
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasRegressor(build_fn=model_builder, epochs=50, batch_size=5, verbose=1)))
pipeline = Pipeline(estimators)
kfold = KFold(n_splits=10, random_state=seed)
results = cross_val_score(pipeline, X_train, Y_train, cv=kfold)
print("Larger: %.2f (%.2f) MSE" % (results.mean(), results.std()))



#model.fit(X_train, y_train, epochs=400)

In [ ]:
model = Sequential()
model.add(Dense(1, input_dim=29, kernel_initializer='normal', activation='relu'))##
#model.add(Dense(6, kernel_initializer='normal', activation='relu'))
#model.add(Dense(1, kernel_initializer='normal'))
# Compile model
model.compile(Adam(lr=0.01), loss='mean_squared_error', metrics=['accuracy'])
model.fit(X_train, Y_train, epochs=50, verbose=1)


In [ ]:
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error as mse

y_train_pred = model.predict(X_train).ravel()
y_test_pred = model.predict(X_test).ravel()

print("The Mean Squared Error on the Train set is:\t{:0.1f}".format(mse(Y_train, y_train_pred)))
print("The Mean Squared Error on the Test set is:\t{:0.1f}".format(mse(Y_test, y_test_pred)))
print("The R2 score on the Train set is:\t{:0.3f}".format(r2_score(Y_train, y_train_pred)))
print("The R2 score on the Test set is:\t{:0.3f}".format(r2_score(Y_test, y_test_pred)))

In [ ]:
model = KerasClassifier(build_fn=model_builder,
                        epochs=50,
                        verbose=1)
cv = KFold(5, shuffle=True)
scores = cross_val_score(model, X, Y, cv=cv)
print("The cross validation accuracy is {:0.4f} ± {:0.4f}".format(scores.mean(), scores.std()))